# Results 

This notebook intends to retrieve results that are in the research paper, information is quite straight forward, and only some configuration tools are setup for better analysis. 

- [Space headway](#space_headway)
- [Time headway](#time_headway) 
- [Acceleration](#acceleration)
- [Trajectories](#trajectories)

Check Bokeh for more details into plot configurations 


In [46]:
import os

from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column, String, Integer, Float 
from sqlalchemy import insert, delete, select, case, and_

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Bokeh
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.sampledata.iris import flowers
from bokeh.io import output_notebook
from bokeh.palettes import Viridis, Spectral8, Reds, Greens, Category20
from bokeh.models import HoverTool, CategoricalColorMapper

output_notebook() 

Loading BokehJS ...

In [47]:
DT = 0.1 # Simulation parameter

KC = 0.16
KH = 0.0896
VF = 25.0
W = 6.25
E  = 25.0*0.3
GCAV = 1/(KC*W)
GHDV = 1/(KH*W)
SCAV = VF/(KC*W)+1/KC
SHDV = VF/(KH*W)+1/KH
dveh_twy = {'CAV': GCAV, 'HDV': GHDV}
dveh_dwy = {'CAV': 1/KC, 'HDV': 1/KH}
U_MAX = 1.5
U_MIN = -1.5

In [48]:
dir_path = os.getcwd()
engine_path = ('..','Output','SymOut.sqlite')
engine_name = os.path.join(os.path.sep,*engine_path)
engine_full_name = os.path.join(dir_path,*engine_path)
engine_call = 'sqlite://'+engine_name
engine = create_engine(engine_call)
metadata = MetaData()

try: 
    ltbstr = 'Loaded table in: '
    connection = engine.connect()    
    traj = Table('traj', metadata, autoload=True, autoload_with=engine)
    closed = Table('closed', metadata, autoload=True, autoload_with=engine)
    headway = Table('headway', metadata, autoload=True, autoload_with=engine)
    control = Table('control', metadata, autoload=True, autoload_with=engine)    
except:
    ltbstr = 'Table could not be loaded: '
finally: 
    print(ltbstr, engine)

Loaded table in:  Engine(sqlite:///../Output/SymOut.sqlite)


In [49]:
stmt = select([closed])
results = connection.execute(stmt).fetchall()
stmt = select([traj])
resultsOL = connection.execute(stmt).fetchall()
column_names = closed.columns.keys()
trajCLDf = pd.DataFrame(results, columns = column_names)
column_names = traj.columns.keys()
trajOLDf = pd.DataFrame(resultsOL, columns = column_names)

In [50]:
trajHDVDf = trajOLDf[trajOLDf['type']=='HDV']

trajCLDf['id'] = trajCLDf['id'].astype('str')
trajHDVDf['id'] = trajHDVDf['id'].astype('str')

sourceCL = ColumnDataSource(trajCLDf)
sourceHDV = ColumnDataSource(trajHDVDf)

CAV_list = trajCLDf.id.unique().tolist()
HDV_list = trajHDVDf.id.unique().tolist()

CAV_colors = [x for i, x in enumerate(Category20[16]) if i % 2 ==0 ]
HDV_colors = [x for i, x in enumerate(Category20[16]) if i % 2 !=0 ]

color_mapper_CAV = CategoricalColorMapper(factors=CAV_list, palette=CAV_colors)
color_mapper_HDV = CategoricalColorMapper(factors=HDV_list, palette=HDV_colors)

colorCAV = dict(field='id', transform=color_mapper_CAV)
colorHDV = dict(field='id', transform=color_mapper_HDV)

hover = HoverTool(tooltips=[('t','@ti'),('x','@abs'),('type','@type'), ('id','@id')])
hoverRef = HoverTool(tooltips=[('t','@ti'), ('id','@id'), ('tau','@tau')])
hoverCtr = HoverTool(tooltips=[('t','@ti'),('u','@ctr'), ('id','@id')])

CAV_colors

/Users/ladino/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['#1f77b4',
 '#ff7f0e',
 '#2ca02c',
 '#d62728',
 '#9467bd',
 '#8c564b',
 '#e377c2',
 '#7f7f7f']

<a id='space_headway'></a>
## Space headway

Head space under control 

In [51]:
p = figure(title = "Space headway", 
           width=800, 
           height=800, 
          tools = ['lasso_select', 'wheel_zoom, reset, save'])

p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'Spacing [m]'

# Spacing 

p.circle('ti', 'spc', source=sourceCL, color = colorCAV, selection_color = colorCAV, size = 3, nonselection_alpha=0.1)
p.add_tools(hover)
show(p)

#
#trajDf.plot(kind='scatter',x='ti',y ='spc', c = trajDf.tron.map(colormap), figsize = (7.5, 7.5), s = 2, grid = True);

In [52]:
stmt = select([headway])
results = connection.execute(stmt).fetchall()
column_names = headway.columns.keys()
headDf = pd.DataFrame(results, columns = column_names)

<a id='time_headway'></a>
## Time headway 

Time headway under control 

In [53]:
# Control performance 

p = figure(title = "Time Gap", 
           width=800, 
           height=800, 
          tools = ['lasso_select', 'wheel_zoom', 'reset', 'save', 'box_zoom'])

p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'Tau [s]'

trajCLDf['tau'] = (trajCLDf['spc']-1/KC).divide(trajCLDf['vit'])
headDf['id'] = headDf['id'].astype('str')
headDf['tau']=headDf['gapt']

REF_list = headDf.id.unique().tolist()

REF_colors = [x for i, x in enumerate(Category20[16]) if i % 2 !=0]

color_mapper_REF = CategoricalColorMapper(factors=REF_list, palette=REF_colors)

sourceTau = ColumnDataSource(trajCLDf)
sourceRef = ColumnDataSource(headDf)

colorREF = dict(field='id', transform=color_mapper_REF)

p.circle('ti', 'tau', source=sourceTau, color = colorCAV, selection_color = colorCAV, size = 3, nonselection_alpha=0.1)
p.circle('ti', 'tau', source=sourceRef, color = colorREF, selection_color = colorREF, size = 3, alpha = 0.2, nonselection_alpha=0.1)
p.add_tools(hoverRef)
show(p)



<a id='acceleration'></a>
## Acceleration signal 

Contorl input for CAVs

In [54]:
stmt = select([control])
results = connection.execute(stmt).fetchall()
column_names = control.columns.keys()
controlDf = pd.DataFrame(results, columns = column_names)

In [55]:
p = figure(title = "Control signal", 
           width=800, 
           height=800, 
          tools = ['lasso_select', 'wheel_zoom, reset, save'])

p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'Acceleration [ms-2]'

controlDf['id'] = controlDf['id'].astype('str')

sourceCTRL = ColumnDataSource(controlDf)

p.circle('ti', 'ctr', source=sourceCTRL, color = colorCAV, selection_color = colorCAV, size = 3, nonselection_alpha=0.1)
p.add_tools(hoverCtr)
show(p)

<a id='trajectories'></a>
## Trajectories 

Space time after control 

In [58]:
p = figure(title = "Trajectories", 
           width=800, 
           height=800, 
          tools = ['lasso_select', 'wheel_zoom', 'reset', 'save','box_zoom'])

p.xaxis.axis_label = 'Time [s]'
p.yaxis.axis_label = 'X [m]'



p.circle('ti', 'abs', source=sourceHDV, color = 'blue', selection_color = colorHDV, size = 1, nonselection_alpha=0.1)
p.circle('ti', 'abs', source=sourceCL, color = 'green', selection_color = colorCAV, size = 1, nonselection_alpha=0.1)
# p.circle('ti', 'abs', source=sourceHDV, color = 'salmon', size = 3)
# p.circle('ti', 'abs', source=sourceCL, color = 'deepskyblue', size = 3)
p.add_tools(hover)

show(p)
